# Basics.
Let's learn to use the BOLT Python API with an exercise. We'll do a simple image classification task on the MNIST dataset. To perform this task, we'll build a fully connected neural network with the following specifications:
* 784 (28 x 28) input dimension
* A single 1000-dim hidden layer with ReLU
* 10-dim output layer with Softmax

In [1]:

from thirdai import bolt

mnist_layers = [
    bolt.LayerConfig(dim=1000, activation_function=bolt.ActivationFunctions.ReLU),
    bolt.LayerConfig(dim=10, activation_function=bolt.ActivationFunctions.Softmax)
]
mnist_network = bolt.Network(layers=mnist_layers, input_dim=784)

====== Building Fully Connected Network ======
Layer: dim=1000, load_factor=1, act_func=ReLU
Layer: dim=10, load_factor=1, act_func=Softmax
Initialized Network in 0 seconds


In [2]:
from thirdai import dataset

print("Loading train dataset...")
mnist_train = dataset.load_bolt_svm_dataset(filename="datasets/mnist/mnist", batch_size=256)

print("Loading test dataset...")
mnist_test = dataset.load_bolt_svm_dataset(filename="datasets/mnist/mnist.t", batch_size=256)


Loading train dataset...
Loading test dataset...
Read 60000 vectors from datasets/mnist/mnist in 0 seconds
Read 10000 vectors from datasets/mnist/mnist.t in 0 seconds


In [3]:
mnist_network.train(train_data=mnist_train, loss_fn=bolt.CategoricalCrossEntropyLoss(), learning_rate=0.001, epochs=1)
mnist_network.predict(test_data=mnist_test, metrics=["categorical_accuracy"], verbose=True)



Epoch 1:
[==================================================] 100%
Processed 235 training batches in 4 seconds
[==================================================] 100%
Processed 40 test batches in 472 milliseconds
Accuracy: 0.9531 (9531/10000)


({'test_time': [472.0], 'categorical_accuracy': [0.9531]},
 array([[1.1310495e-11, 8.2258145e-10, 5.8272608e-06, ..., 9.9998641e-01,
         1.2942891e-10, 3.0245258e-07],
        [4.9792249e-07, 2.9483358e-03, 9.9565291e-01, ..., 2.2643784e-12,
         3.3942335e-06, 5.6468863e-10],
        [1.0083226e-07, 9.9855953e-01, 2.6424002e-04, ..., 3.3749739e-04,
         2.1043376e-04, 2.4309951e-05],
        ...,
        [1.0667796e-09, 9.4360580e-08, 3.3615845e-07, ..., 1.3852460e-05,
         7.6349727e-03, 1.7256956e-03],
        [4.3771325e-07, 4.2658760e-07, 1.1811041e-07, ..., 5.4442144e-09,
         1.6028654e-04, 3.0941254e-09],
        [4.3474174e-10, 6.4210627e-13, 2.4786304e-09, ..., 1.4204260e-13,
         2.4888418e-11, 4.7382248e-10]], dtype=float32))

# Can we get wider?

In [10]:
bigger_layers = [
    bolt.LayerConfig(dim=10000, load_factor=0.1, activation_function=bolt.ActivationFunctions.ReLU),
    bolt.LayerConfig(dim=151, activation_function=bolt.ActivationFunctions.Softmax)
]
bigger_network = bolt.Network(layers=bigger_layers, input_dim=5512)

====== Building Fully Connected Network ======
Layer: dim=10000, load_factor=1, act_func=ReLU
Layer: dim=151, load_factor=1, act_func=Softmax
Initialized Network in 0 seconds


In [8]:

print("Loading train dataset...")
intent_class_train = dataset.load_bolt_svm_dataset(filename="datasets/intent_classification/train_shuf.svm", batch_size=256)

print("Loading test dataset...")
intent_class_test = dataset.load_bolt_svm_dataset(filename="datasets/intent_classification/test_shuf.svm", batch_size=256)

Loading train dataset...
Read 18100 vectors from datasets/intent_classification/train_shuf.svm in 2 seconds
Loading test dataset...
Read 5500 vectors from datasets/intent_classification/test_shuf.svm in 0 seconds


In [ ]:
bigger_network.train(train_data=intent_class_train, loss_fn=bolt.CategoricalCrossEntropyLoss(), learning_rate=0.001, epochs=2)
bigger_network.enable_sparse_inference()
bigger_network.train(train_data=intent_class_train, loss_fn=bolt.CategoricalCrossEntropyLoss(), learning_rate=0.001, epochs=1)
bigger_network.predict(test_data=intent_class_test, metrics=["categorical_accuracy"], verbose=True)


Epoch 1:
[==================================================] 100%
Processed 71 training batches in 80 seconds

Epoch 2:
[==                                                ] 4%

# Let's get wider.

In [ ]:
sentiment_analysis_layers = [
    bolt.LayerConfig(dim=2000, 
        load_factor=0.2, 
        activation_function=bolt.ActivationFunctions.ReLU,
        sampling_config=bolt.SamplingConfig(
            hashes_per_table=4,
            num_tables=64,
            range_pow=4 * 3,
            reservoir_size=64,
        )),
    bolt.LayerConfig(dim=2,
        load_factor=1.0, 
        activation_function=bolt.ActivationFunctions.Softmax,
        )     
]
sentiment_analysis_network = bolt.Network(layers=sentiment_analysis_layers, input_dim=100000)
sentiment_analysis_network.save("pathtosavefile")

In [ ]:
sentiment_analysis_network = bolt.Network.load("pathtosavefile")

import interactive_sentiment_analysis
interactive_sentiment_analysis.demo(sentiment_analysis_network)